# Computational Hydrogen Electrode Tutorial

Import useful modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read
from ase import Atoms
from ase.build import mx2, molecule
from ase.constraints import FixAtoms
from ase.visualize import view
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import QuasiNewton
import time
from ase.units import kB

Use ASE to generate the material. We will use a 2x2 supercell with 10 Angstrom of vacuum

In [ ]:
MoS2 = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(2, 2, 1), vacuum=10.)

In [ ]:
MoS2.positions

Setup of QE command

In [ ]:
runprefix = "mpirun -np 4 "
qepath = "/Users/oliviero/PWSCF/espresso-git/bin/"
pseudodir = "./pseudos"

In [ ]:
# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command=runprefix+qepath+'pw.x', pseudo_dir=pseudodir
)

In [ ]:
pseudopotentials = {
    "H":"H.pbe-rrkjus_psl.1.0.0.UPF",
    "O":"O.pbe-n-kjpaw_psl.0.1.UPF",
    "Mo":"Mo_ONCV_PBE-1.0.oncvpsp.upf",
    "S":"s_pbe_v1.4.uspp.F.UPF"
}

## MoS2 Optimization

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2.calc = calc

We don't want to run this command, we want to do a proper optimization of the cell

In [ ]:
eMoS2 = MoS2.get_potential_energy()

In [ ]:
print(eMoS2)

We can read the pre-run vc relax output here

In [ ]:
MoS2 = read('./references/MoS2_vcrelax.out')

In [ ]:
MoS2.center()

In [ ]:
MoS2.positions

We can re-compute the energy of the system in ASE now. 

In [ ]:
MoS2.calc = calc
eMoS2 = MoS2.get_potential_energy()
print(eMoS2)

## Setup Adsorbate Calculations

Let us consider asdorption on a sulfur atom (any of the S atoms work)

In [ ]:
atop_index = 1

In [ ]:
MoS2.positions[atop_index]

To speed up the calculations, let impose that all the other atoms of the system are frozen

In [ ]:
fixed = list(range(len(MoS2)))
fixed.remove(atop_index)
print(fixed)
constraint = FixAtoms(indices=fixed)
MoS2.set_constraint(constraint)

## Adsorbing OH

In [ ]:
# Creating the OH molecule
oh_molecule = Atoms('OH', positions=[(0, 0, 0), (0, -0.763, 0.596)])

oh_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.87))

oh_molecule.positions

In [ ]:
MoS2OH = MoS2 + oh_molecule
MoS2OH.positions

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2OH'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2OH.calc = calc

In [ ]:
dyn = QuasiNewton(MoS2OH, trajectory='MoS2OH.traj')
t = time.time()
# dyn.run(fmax=0.05)
MoS2OH = read('./references/MoS2OH.traj')
print('Calculation time: {} min.'.format((time.time() - t) / 60))

In [ ]:
MoS2OH.positions

In [ ]:
eMoS2OH = MoS2OH.get_potential_energy()
print(eMoS2OH)

In [ ]:
print("bond S-O: ", MoS2OH.get_distance(1,12))
print("bond O-H: ", MoS2OH.get_distance(12,13))

## Adsorbing O 

In [ ]:
# Creating the OH molecule
o_molecule = Atoms('O', positions=[(0, 0, 0)])

o_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.872))

o_molecule.positions

In [ ]:
MoS2O = MoS2 + o_molecule
MoS2O.positions

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2O'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2O.calc = calc

In [ ]:
dyn = QuasiNewton(MoS2O, trajectory='MoS2O.traj')
t = time.time()
#dyn.run(fmax=0.05)
MoS2O = read('./references/MoS2O.traj')
print('Calculation time: {} min.'.format((time.time() - t) / 60))

In [ ]:
eMoS2O = MoS2O.get_potential_energy()
print(eMoS2O)

In [ ]:
print("bond S-O: ", MoS2O.get_distance(1,12))

## Adsorbing OOH

In [ ]:
ooh_molecule = Atoms('OOH', positions=[(0, 0, 0), (0.75, 0.34, 1.10), (0.1, 0.25, 1.85)])

ooh_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.87))

ooh_molecule.positions

In [ ]:
MoS2OOH = MoS2 + ooh_molecule
MoS2OOH.positions

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2O'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2OOH.calc = calc

In [ ]:
dyn = QuasiNewton(MoS2OOH, trajectory='MoS2OOH.traj')
t = time.time()
#dyn.run(fmax=0.05)
MoS2OOH = read('./references/MoS2OOH.traj')
print('Calculation time: {} min.'.format((time.time() - t) / 60))

In [ ]:
eMoS2OOH = MoS2OOH.get_total_energy()
print(eMoS2OOH)

## Isolated Molecules (H2 and H2O)

In [ ]:
H2_molecule = molecule("H2")
H2_molecule.set_cell(MoS2.get_cell())
H2_molecule.calc = calc
dynH2 = QuasiNewton(H2_molecule, trajectory='H2.traj')
t = time.time()
#dynH2.run(fmax=0.05)
H2 = read('./references/H2.traj')
print('Calculation time: {} min.'.format((time.time() - t) / 60))
eH2 = H2_molecule.get_potential_energy()
print("H2 energy: ", eH2, " eV")
print("H-H bond lenght: ", H2_molecule.get_distance(0,1))

In [ ]:
H2O_molecule = molecule("H2O")
H2O_molecule.set_cell(MoS2.get_cell())
H2O_molecule.calc = calc
dynH2O = QuasiNewton(H2O_molecule, trajectory='H2O.traj')
t = time.time()
#dynH2O.run(fmax=0.05)
H2O = read('./references/H2O.traj')
print('Calculation time: {} min.'.format((time.time() - t) / 60))
eH2O = H2O_molecule.get_potential_energy()
print("H2O energy: ", eH2O, " eV")
print("O-H bond lenght: ", H2O_molecule.get_distance(0,1))

In [ ]:
print(eMoS2OH,eH2O,eMoS2)

## Some Thermodynamics

These come from experiments (are we really doing first-principles?)

In [ ]:
DGtot = 4.92
zpeH2O = 0.57
zpeH2 = 0.35

In [ ]:
tdsH2O = 0.67
tdsH2 = 0.403
kBT = kB * 298

These can supposedly come from vibrational calculations

In [ ]:
zpeMoS2O = 0.06
zpeMoS2OH = 0.37
zpeMoS2OOH = 0.44
zpeMoS2 = 0.

Compute free energy differences of individual steps

In [ ]:
DE1_SHE = eMoS2OH - eH2O - eMoS2 + 0.5*eH2
DZPE1 = zpeMoS2OH - zpeH2O - zpeMoS2 + 0.5*zpeH2
TDS1 = - tdsH2O + 0.5*tdsH2
DG1_SHE = DE1_SHE + DZPE1 + TDS1
print(DG1_SHE)

In [ ]:
DE2_SHE = eMoS2O - eMoS2OH + 0.5*eH2
DZPE2 = zpeMoS2O - zpeMoS2OH + 0.5*zpeH2
TDS2 = 0.5*tdsH2
DG2_SHE = DE2_SHE + DZPE2 + TDS2
print(DG2_SHE)

In [ ]:
DE3_SHE = eMoS2OOH - eMoS2O - eH2O + 0.5*eH2
DZPE3 = zpeMoS2OOH - zpeMoS2O - zpeH2O + 0.5*zpeH2
TDS3 = -tdsH2O + 0.5*tdsH2
DG3_SHE = DE3_SHE + DZPE3 + TDS3
print(DG3_SHE)

In [ ]:
DG4_SHE = DGtot - DG1_SHE - DG2_SHE - DG3_SHE
print(DG4_SHE)

What happens at 0 applied potential wrt SHE?

In [ ]:
steps = np.array([-0.5,0.5, 1.5, 2.5, 3.5, 4.5])
dgs_she = np.array([0,0, DG1_SHE, DG2_SHE, DG3_SHE, DG4_SHE])
plt.step(steps,np.cumsum(dgs_she))
plt.xlabel('Reaction Step',fontsize=20)
plt.ylabel('$\Delta$G (eV)',fontsize=20)

What happens at the official redox potential for oxygen evolution (U=1.23V vs SHE)?

In [ ]:
pH = 0.
U = 4.92/4
DG1_U = DG1_SHE - kBT*np.log(10)*pH - U
DG2_U = DG2_SHE - kBT*np.log(10)*pH - U
DG3_U = DG3_SHE - kBT*np.log(10)*pH - U
DG4_U = DG4_SHE - kBT*np.log(10)*pH - U


In [ ]:
steps = np.array([-0.5,0.5, 1.5, 2.5, 3.5, 4.5])
dgs_she = np.array([0,0, DG1_SHE, DG2_SHE, DG3_SHE, DG4_SHE])
dgs_U = np.array([0,0, DG1_U, DG2_U, DG3_U, DG4_U])
plt.step(steps,np.cumsum(dgs_she))
plt.step(steps,np.cumsum(dgs_U))
plt.xlabel('Reaction Step',fontsize=20)
plt.ylabel('$\Delta$G (eV)',fontsize=20)

What is the overpotential?

In [ ]:
eta = np.max(dgs_she[2:] - 4.92/4)

In [ ]:
pH = 0.
U = 4.92/4 + eta
DG1_U = DG1_SHE - kBT*np.log(10)*pH - U
DG2_U = DG2_SHE - kBT*np.log(10)*pH - U
DG3_U = DG3_SHE - kBT*np.log(10)*pH - U
DG4_U = DG4_SHE - kBT*np.log(10)*pH - U

In [ ]:
steps = np.array([-0.5,0.5, 1.5, 2.5, 3.5, 4.5])
dgs_she = np.array([0,0, DG1_SHE, DG2_SHE, DG3_SHE, DG4_SHE])
dgs_Ueta = np.array([0,0, DG1_U, DG2_U, DG3_U, DG4_U])
plt.step(steps,np.cumsum(dgs_she))
plt.step(steps,np.cumsum(dgs_U))
plt.xlabel('Reaction Step',fontsize=20)
plt.ylabel('$\Delta$G (eV)',fontsize=20)